In [ ]:
## Author: Abhishek Sinha, Tata Institute of Fundamental Research, Mumbai, India
## All rights reserved

## This code is meant to be run on Google Colab

from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive')
import numpy as np
import pandas as pd
from pandas.core.common import flatten
np.random.seed(16) ## e.g., put seed=7
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
device = torch.device('cpu') ## Running on cpu


In [ ]:
class COCO:
    def __init__(self, n, T, lambda_param): ### n denotes the dimension of input
        #self.D=np.sqrt(2) ### D denotes the Euclidean diameter of the feasible set, which is sqrt(2) for simplex
        self.D=10 ### Diameter of the action set
        self.n=n
        self.x=np.array(proj(np.random.rand(n))).reshape(-1,1) ### n denotes the input dimension
        #self.x=np.array(proj(np.ones(n)/n)).reshape(-1,1)
        self.step_size=0
        self.grad_sum_sq=0.001 ### internal variable needed to compute the step size
        self.sum_x=self.x ### internal variable required to get the average prediction
        self.Q=0 ### initializing the cumulative constraint violation
        self.TotalCost=0 ### initializing the total cost incurred
        #self.Lambda=1.0/(2.0*np.sqrt(T)) ## the default setting
        #self.Lambda=0.038 ## custom setting for the credit card fraud detection dataset
        self.Lambda=lambda_param
        self.V=1   ## setting the required hyper parameters




    def predict_COCO(self, grad):
        self.x=np.array(proj(self.x-self.step_size*grad)).reshape(-1,1)
        return

    def update_COCO(self, cost_val, constr_val):
        self.Q = self.Q+constr_val
        self.TotalCost=self.TotalCost+cost_val ## updating CCV and cost
        self.sum_x=self.sum_x+self.x

    def surrogate_cost_grad(self, cost_grad, constr_grad): # returns the gradient of the surrogate cost function at x
        grad=self.V*cost_grad.reshape(-1,1)+self.Lambda*np.exp(self.Lambda*self.Q)*constr_grad.reshape(-1,1)
        self.grad_sum_sq=self.grad_sum_sq+np.linalg.norm(grad)**2
        self.step_size=self.D/np.sqrt(2.0*self.grad_sum_sq) ### selecting step sizes using AdaGrad
        return grad.reshape(-1,1)





In [ ]:
class NN:
    ## implements the case when the cost and constraint functions are given by a Neural Network
    def __init__(self):
        self.id='NN'
        self.problem_class='classification'
        self.t=-1 ### counter indicates the current round number
        self.MISS=0 ## counter to keep track of number of misses
        self.N1=0 ## count of examples belonging to class 1
        self.FA=0 ### counter to keep track of number of FA
        self.N0=0 ## count of examples belonging to class 0

        self.data=pd.read_csv('/content/drive/MyDrive/creditcard.csv')
        self.data=self.data.sample(frac=1) ## randomly shuffling the rows
        self.z=np.array(self.data.iloc[:,1:30]) ## z denotes the feature vector matrix
        self.y=np.array(self.data['Class'])    ## y denotes the column of labels

        ##--------------------

        object= StandardScaler()
        self.z= object.fit_transform(self.z) ## preprocessing the feature vectors

        #### -------------
        ## configuring the NN architecture and the loss function

        self.N, self.D_in, self.H, self.D_out = 1, self.z.shape[1], 10, 1 ## 10 hidden layer width, 4 dimensional feature

        self.model = torch.nn.Sequential(
          torch.nn.Linear(self.D_in, self.H),
          #torch.nn.ReLU(),
          torch.nn.Sigmoid(),
          #torch.nn.Linear(self.H, self.H),
          #torch.nn.ReLU(),
          torch.nn.Linear(self.H, self.D_out),
          torch.nn.Sigmoid(), ## the output layer
        ).to(device)

        self.loss_fn = torch.nn.BCELoss() ## remember - it is simply the negative log likelihood - training via Max likelihood

    def update_grad(self):
        self.t+=1
        self.model.zero_grad()
        self.hat_y=self.model(torch.tensor(self.z[self.t,:]).to(torch.float32)) ## predicted value
        self.hat_y=torch.clamp(self.hat_y, min=1e-7, max=1-1e-7) ## avoiding extreme values for numerical stability
        self.loss=self.loss_fn(torch.squeeze(self.hat_y), torch.tensor(self.y[self.t]).to(torch.float32))
        self.loss.backward()
        gradients = []
        for param in self.model.parameters():
            if param.grad is not None:
                gradients.append(param.grad.view(-1))
        self.grad_vector = np.array(torch.cat(gradients))

        ## Updating the performance metrics
        self.N0+=(1-self.y[self.t])
        self.FA+=(1-self.y[self.t])*self.hat_y
        self.N1+=self.y[self.t]
        self.MISS+=self.y[self.t]*(1-self.hat_y)


    def grad_f(self, x):
        if(self.y[self.t]==0):
            return self.grad_vector
        else:
            return np.zeros(len(self.grad_vector))


    def grad_g(self, x):
        if(self.y[self.t]==1):
            return self.grad_vector
        else:
            return np.zeros(len(self.grad_vector))



    def f_val(self, x):
        return -(1-self.y[self.t])*torch.log(1.0-self.hat_y) ## f= -(1-y)log(1-y_hat) (Liklihood value)


    def g_val(self, x):  ## g = -y log(y_hat) assuming bias=0
        return -self.y[self.t]*(min(0.0, torch.log(self.hat_y)-torch.log(torch.tensor(1.0))))



## Assigning the parameters returned by COCO to the NN
def assign_parameters(model, vector):
    """
    Assign the parameters of a PyTorch neural network using the given vector.

    Args:
    - model (nn.Module): The PyTorch neural network model.
    - vector (list or numpy array or torch tensor): The vector containing the new parameters.

    Returns:
    - None
    """
    if not isinstance(vector, torch.Tensor):
        vector = torch.tensor(vector, dtype=torch.float32)

    # Ensure the vector has the same number of elements as the model's parameters
    params = torch.cat([p.view(-1) for p in model.model.parameters()])
    assert params.numel() == vector.numel(), "Vector size must match the number of model parameters"

    # Assign the parameters
    index = 0
    for p in model.model.parameters():
        numel = p.numel()
        p.data = vector[index:index+numel].view(p.size())
        index += numel




In [ ]:
### Driver code

## It might need a few hours to finish execution depending on the number of different lambda parameter settings to be tested in
## the hyper_parameter_lambda variable below. If necessary, reduce this number.
#---------------------------

hyper_parameter_lambda=np.linspace(0.01,0.05,30)
ROC=np.zeros([0,2])
for lambda_param in hyper_parameter_lambda: # sweep of the hyper-parameter lambda
    adversary=NN()

    if adversary.id=='2Player':
        n=100
        T=100000
        from projection import projection_simplex_bisection as proj ### loading the simplex projection module
    elif adversary.id=='banknote':
        T, n=adversary.z.shape
        from projection import euclidean_projection as proj ### loading the Euclidean projection module
    elif adversary.id=='linsep':
        T, n=adversary.z.shape
        from projection import euclidean_projection as proj ### loading the Euclidean projection module
    elif adversary.id=='iris':
        T, n=adversary.z.shape
        from projection import euclidean_projection as proj ### loading the Euclidean projection module
    elif adversary.id=='NN':
        T=adversary.z.shape[0]
        grad_vec=[]
        for param in adversary.model.parameters():
            grad_vec.append(np.array(param.data))
        n=len(list(flatten(grad_vec))) ## counting the number of parameters
        from projection import euclidean_projection as proj ### loading the Euclidean projection module
    ##########################if


    coco = COCO(n,T, lambda_param) ### Algorithm object


    cost_vec=np.zeros([1,T])
    violation_vec=np.zeros([1,T]) ### Arrays to store the cumulative cost and violation incurred by COCO
    regret_vec=np.zeros([1,T])

    for t in range(T):
        current_action=coco.x # getting the current action of the algorithm

        ## assigning the parameters to the NN object

        if adversary.id=='NN':
            assign_parameters(adversary, current_action)
            adversary.update_grad()

        ## Calling adversary to get its choice of cost and constraint functions
        cost_grad=adversary.grad_f(current_action)
        constr_grad=adversary.grad_g(current_action)
        cost_val=adversary.f_val(current_action)
        constr_val=adversary.g_val(current_action)  ### getting adversary's choices


        if adversary.id=='NN':
            constr_val=constr_val.item()

        ## Calling COCO to update its internal states and to determine its next action
        coco.update_COCO(cost_val, constr_val) ### updating internal states of the algorithm
        surrogate_cost_grad=coco.surrogate_cost_grad(cost_grad, constr_grad) ### computing the gradient of the surrogate cost

        coco.predict_COCO(surrogate_cost_grad)  ### predicting the next_action

        #### recording the statistics
        cost_vec[0,t]=coco.TotalCost
        violation_vec[0,t]=coco.Q
        if adversary.problem_class=='game':
            regret_vec[0,t]=coco.TotalCost - min(adversary.cost_grad_sum) ## this is a little erroneous as it computes regret over the entire admissible set

     #### computing the number of classification errors
    if adversary.problem_class=='classification':
        #print('False Positive rate:',  adversary.FA/adversary.N0, 'False Negative rate:', adversary.MISS/adversary.N1)
        FP=adversary.FA/adversary.N0
        FN=adversary.MISS/adversary.N1
        FPR=FP
        TPR=1-FN
        new_rate=[FPR, TPR]
        # Convert each tensor to a NumPy array
        numpy_array_list = [tensor.detach().numpy() for tensor in new_rate]
        rate_array = np.concatenate(numpy_array_list)
        print(rate_array)
        ROC = np.vstack([ROC, rate_array])


# Save the ROC values to a file for plotting the data
np.save("ROC_data2.dat", ROC)


In [ ]:
## Plots the pre-computed result stored in data/ROC_data. If you want to plot the above results, simply replace this
## file with the new version
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False
import pandas as pd
import numpy as np

#plt.rcParams['text.usetex'] = True
#sns.set(style='ticks', palette='Set2')
sns.set()

sns.set_theme("paper")
sns.set_style("dark")
ROC=pd.read_csv('/content/drive/MyDrive/data/ROC_data', sep='\s+')
ROC=ROC.sort_values('FPR')
plt.axis([0.01, 1.0, 0.6, 1])

AUC = np.trapz(ROC['TPR'], ROC['FPR'])
print("Area under the ROC curve is:", AUC)

plt.ylabel("True Positive Rate", fontsize=15)
plt.xlabel("False Positive Rate", fontsize=15)
plt.plot(ROC['FPR'], ROC['TPR'], linewidth = 2.0)
plt.grid(linestyle = '--', linewidth = 0.7)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.suptitle('ROC curve', fontsize=20)
plt.fill_between(ROC['FPR'], ROC['TPR'], color='lightsteelblue', alpha=0.5, label=f'Area under the ROC curve= {AUC:.2f}')
plt.legend(loc='lower right', fontsize=14)
plt.savefig('ROC_plt.pdf', dpi=1000)
